# Unitary Decomposition Based on QuCT

**Author:** Congliang Lang & Siwei Tan  

**Date:** 8/4/2024

Based on paper "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features][1]" (MICRO 2023)

[1]: https://dl.acm.org/doi/10.1145/3613424.3614274

The expressiveness of gate vectors enables QuCT to reconstruct sub-circuits via gate vectors, which is a necessary ability for tasks involving circuit generation, such as the unitary decomposition. Unitary decomposition is a task that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.

In [1]:
import os
os.chdir("../..")
# import sys
# sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

import time
import ray
ray.init(log_to_driver=False)
# ray.init(log_to_driver=False, runtime_env={"working_dir": "../"})
from qiskit.quantum_info import random_unitary

from janusq.objects.circuit import qiskit_to_circuit
from janusq.objects.random_circuit import random_circuits
from janusq.objects.backend import  LinearBackend
from janusq.analysis.vectorization import RandomwalkModel
from janusq.analysis.unitary_decompostion import decompose

2025-02-08 15:35:35,981	INFO worker.py:1724 -- Started a local Ray instance.


## Construct U2V model

The U2V model serves as the bridge between unitaries and gate vectors, where the sub-circuits reconstructed from these candidate vectors will replace the search space of QFAST. To build such a model, we obtain a U2V dataset composed of <$unitary,~\{vectors\}$> pairs, derived from a set of random circuits generated with the same scheme mentioned in Section~\ref{sec:equ_prediction}.

In [2]:
n_qubits = 4
backend = LinearBackend(n_qubits, 1, basis_two_gates = ['crz'])

In [3]:
from janusq.analysis.unitary_decompostion import U2VModel

n_step = 2

dataset = random_circuits(backend, n_circuits=50, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=False)

up_model = RandomwalkModel(
    n_step, 4 ** n_step, backend, directions=('parallel', 'next'))
up_model.train(dataset, multi_process=True, remove_redundancy=False)

u2v_model = U2VModel(up_model)
data = u2v_model.construct_data(dataset, multi_process=False)
u2v_model.train(data, n_qubits)


100%|██████████| 6/6 [00:00<00:00,  8.48it/s]


len(Us) =  1005 len(gate_vecs) =  1005
Start construct U2VMdoel
Finish construct U2VMdoel, costing 0.7678520679473877s


## Decompose a Unitary

In [4]:
# generate a random unitary
unitary = random_unitary(2**n_qubits).data

# apply decomposition
start_time = time.time()
quct_circuit = decompose(unitary, allowed_dist = 0.5, backend = backend, u2v_model = u2v_model, multi_process = True)
quct_time = time.time() - start_time

print(quct_time)
print(quct_circuit)

108.71477198600769
      ┌─────────────────────────────┐  ░                               ░ »
q_0: ─┤ U(-1.9834,0.77051,-0.49156) ├──░──────────────────────■────────░─»
      └┬────────────────────────────┤  ░                      │        ░ »
q_1: ──┤ U(0.74593,-1.4629,-1.0038) ├──░────────■─────────────┼────────░─»
     ┌─┴────────────────────────────┴┐ ░ ┌──────┴──────┐      │        ░ »
q_2: ┤ U(-0.74855,-0.99611,-0.22433) ├─░─┤ Rz(-2.6033) ├──────┼────────░─»
     └─┬───────────────────────────┬─┘ ░ └─────────────┘┌─────┴──────┐ ░ »
q_3: ──┤ U(2.1633,0.84052,-1.9665) ├───░────────────────┤ Rz(-3.415) ├─░─»
       └───────────────────────────┘   ░                └────────────┘ ░ »
«     ┌─────────────────────────────┐ ░              ░ »
«q_0: ┤ U(0.95749,0.092231,0.77051) ├─░──────■───────░─»
«     └┬────────────────────────────┤ ░      │       ░ »
«q_1: ─┤ U(0.21615,1.6227,-0.53666) ├─░──────┼───────░─»
«      ├───────────────────────────┬┘ ░ ┌────┴─────┐ ░ »
«q_2: ─┤ U(1.9789,-1.

In [5]:
# compare it with the qsd method
from qiskit.synthesis.unitary.qsd import qs_decomposition

start_time =time.time()
qc = qs_decomposition(unitary)

qsd_circuit = qiskit_to_circuit(qc)
qsd_time = time.time() - start_time

print(qsd_time)
print(qc)

0.05326509475708008
global phase: 3.9962
       ┌──────────────┐                                                   »
q3_0: ─┤0             ├───■────────────────────────────────────────────■──»
       │  circuit-429 │   │                                            │  »
q3_1: ─┤1             ├───┼──────────────────────■─────────────────────┼──»
      ┌┴──────────────┴┐┌─┴─┐┌────────────────┐┌─┴─┐┌───────────────┐┌─┴─┐»
q3_2: ┤ U(0,0,-0.5056) ├┤ X ├┤ U(0,0,0.73944) ├┤ X ├┤ U(0,0,1.3202) ├┤ X ├»
      ├────────────────┤└───┘└────────────────┘└───┘└───────────────┘└───┘»
q3_3: ┤ U(0,0,0.44458) ├──────────────────────────────────────────────────»
      └────────────────┘                                                  »
«                              ┌──────────────┐                   »
«q3_0: ────────────────────────┤0             ├────────────────■──»
«                              │  circuit-438 │                │  »
«q3_1: ────────────────────■───┤1             ├────────────────┼──»
«  

/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [6]:
synthesis_method_result = [qsd_circuit,  quct_circuit]
synthesis_method_time = [qsd_time,  quct_time]
for res, tim in zip(synthesis_method_result, synthesis_method_time):
    print(f"#gate: {res.n_gates}, #two_qubit_gate: {res.num_two_qubit_gate}, depth: {res.depth}, time: {tim} \n")

#gate: 193, #two_qubit_gate: 83, depth: 175, time: 0.05326509475708008 

#gate: 32, #two_qubit_gate: 10, depth: 14, time: 108.71477198600769 

